In [ ]:
#our fork of the Tessellate-Imaging image detection library
#!rm -rf Monk_Object_Detection
! git clone https://github.com/roboflow-ai/Monk_Object_Detection.git

Cloning into 'Monk_Object_Detection'...
remote: Enumerating objects: 3794, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 3794 (delta 28), reused 31 (delta 16), pack-reused 3747
Receiving objects: 100% (3794/3794), 132.21 MiB | 33.47 MiB/s, done.
Resolving deltas: 100% (828/828), done.
Updating files: 100% (4032/4032), done.


In [ ]:
# For colab use the command below
# Set up library requirments
! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

xargs: warning: options --max-args and -L are mutually exclusive, ignoring previous --max-args value
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
  Cloning https://github.com/abhi-kumar/cocoapi.git to /tmp/pip-install-ki6n949q/pycocotools_5093f7e3423f4a45b03ca234894a7be4
  Running command git clone --filter=blob:none --quiet https://github.com/abhi-kumar/cocoapi.git /tmp/pip-install-ki6n949q/pycocotools_5093f7e3423f4a45b03ca234894a7be4
  Resolved https://github.com/abhi-kumar/cocoapi.git to commit 1d6d019f8938f47a9b5af28685011898b25ff93a
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=375587 sha256=f56990e943f66e93ecb1ddc720108200a7a3dae560b3f25e3317c96dff31747a
  Stored in directory: /tmp/pip-ephem-wheel-

In [ ]:
#fixed version of tqdm output for Colab
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
#IGNORE restart runtime warning, it is indeed installed
#missing a few extra packages that we will need later!
!pip install efficientnet_pytorch
!pip install tensorboardX

     - 91.8 kB 1.7 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tqdm: filename=tqdm-4.28.1-py2.py3-none-any.whl size=47874 sha256=94816ef45a8f71a8bb669fc764615bd68fe238b613bfb00bb950def5ad6aa021
  Stored in directory: /tmp/pip-ephem-wheel-cache-dc1utv05/wheels/65/77/d5/d5ddeac9924f01d101ed3d2bf420c627eba535f8b8d93f27ee
Successfully built tqdm
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
huggingface-hub 0.20.3 requires tqdm>=4.42.1, but you have tqdm 4.28.1 which is incompatible.
panel 1.3.8 requires tqdm>=4.48.0, but you have tqdm 4.28.1 which is incompatible.
prophet 1.1.5 requires tqdm>=4.36.1, but you have tqdm 4.28.1 which is incompatible.
spacy 3.7.4 requires tqdm<5.0.0,>=4.38.0, 

# Let's get some data!

The best part about Roboflow is the efficient management of your datasets. [Upload you dataset](roboflow.ai) and you will recieve a fresh curl code to ouput it in whatever augmented and annotated format you need.

In [ ]:
#fresh curl courtesy of roboflow.ai, outputing our dataset in Coco Json format

!curl -L https://app.roboflow.com/ds/jI4y3gVNIG?key=Vp29Wwh00g > Thermal.zip; unzip Thermal.zip; rm Thermal.zip
#!curl -L https://app.roboflow.com/ds/2DfyUuDtBS?key=TGR0qRTzOS > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   2291      0 --:--:-- --:--:-- --:--:--  2286
100 36.9M  100 36.9M    0     0  29.1M      0  0:00:01  0:00:01 --:--:-- 29.1M
Archive:  Thermal.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/FLIR_00001_jpeg.rf.a42cf162429deaab47b495ad8e72d06e.jpg  
 extracting: test/FLIR_00130_jpeg.rf.398205f06fcfd79052ce522c9f36a0e5.jpg  
 extracting: test/FLIR_00164_jpeg.rf.ae26b9b630e66a545b6194077727d0f4.jpg  
 extracting: test/FLIR_00176_jpeg.rf.79b89afe7c3fb0c921f7d85d904b75cc.jpg  
 extracting: test/FLIR_00187_jpeg.rf.b823d5fe0fadb68a6d55d2121817135f.jpg  
 extracting: test/FLIR_00220_jpeg.rf.f0d4c8cd5522c4ba29cb9b714c976210.jpg  
 extracting: test/FLIR_00244_jpeg.rf.292272eae20938a108ab3ec53d3cf5e5.jpg  
 extracting: test/FLIR_00262_jpeg.r

In [ ]:
#let's take a look at our directory
#notice the data came down in train, valid, test, splits - this is pre set during the dataset upload process
%ls

Monk_Object_Detection/  README.roboflow.txt  test/   valid/
README.dataset.txt      sample_data/         train/


In [ ]:
#let's take a peak in train
#jpg images and some coco json annotations
%ls train

_annotations.coco.json
FLIR_00002_jpeg.rf.60ae0d6cc664490e44969b43212dc506.jpg
FLIR_00004_jpeg.rf.52ef4da34ada20e1cd58b588ffe31eae.jpg
FLIR_00005_jpeg.rf.e9816cb5076cd6a713f266ed2f0d4607.jpg
FLIR_00006_jpeg.rf.a17c5c653b17159431efb873b5d59213.jpg
FLIR_00013_jpeg.rf.ef529f8e469838f23b946333bc31fec9.jpg
FLIR_00016_jpeg.rf.9541b0b1895030d17690ece73b91a07e.jpg
FLIR_00018_jpeg.rf.fed7778341fd37042f6e2a58320a1dc4.jpg
FLIR_00024_jpeg.rf.96ebee135d7978c3b0306e0bb25b2a73.jpg
FLIR_00030_jpeg.rf.f8a3f50cc7976e407e5e213a928e39cf.jpg
FLIR_00038_jpeg.rf.7d99ffba549f3908466120d136f14703.jpg
FLIR_00042_jpeg.rf.88f3dabebc247e2937be99b2f45a5eff.jpg
FLIR_00045_jpeg.rf.40c7844089569d86c33ed2ff7c4fae78.jpg
FLIR_00046_jpeg.rf.0c8637b7f718723d3f18504cfadfacf5.jpg
FLIR_00047_jpeg.rf.47498e4a9239e4c05b38bc8bc4f26ea9.jpg
FLIR_00087_jpeg.rf.e7ce4185c50e0213fa7ad677d1063c94.jpg
FLIR_00088_jpeg.rf.e2c82783e81304e6c9a2505c5e52da00.jpg
FLIR_00089_jpeg.rf.c0bca48a35973dc68421b945ff4df1c9.jpg
FLIR_00097_jpeg.rf.e706fa

In [ ]:
#in the next three cells, we move the data into a structure that the image detection library will be expecting
#but no file data manipulation is necessary
#images can also be segmented into class folders, but we combine all classes here
!mkdir Cars
!mkdir Cars/annotations
!mkdir Cars/Annotations
!mkdir Cars/Images

In [ ]:
%cp train/_annotations.coco.json Cars/annotations/instances_Images.json

In [ ]:
%cp train/*.jpg Cars/Images/

# Training

In this section we set up the efficientDet-d0 model from backbone and train to our custom case

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/4_efficientdet/lib/");

In [ ]:
from train_detector import Detector

In [ ]:
gtf = Detector();

In [ ]:
#directs the model towards file structure
root_dir = "./";
coco_dir = "Cars";
img_dir = "./";
set_dir = "Images";

In [ ]:
#smells like some free compute from Colab, nice
gtf.Train_Dataset(root_dir, coco_dir, img_dir, set_dir, batch_size=8, image_size=512, use_gpu=True)

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:

gtf.Model();

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████████████████████████████████████████████████████| 20.4M/20.4M [00:00<00:00, 147MB/s]


Loaded pretrained weights for efficientnet-b0


In [ ]:
gtf.Set_Hyperparams(lr=0.0001, val_interval=1, es_min_delta=0.0, es_patience=0)

In [ ]:
%%time
gtf.Train(num_epochs=18, model_output_dir="trained/");

/content/Monk_Object_Detection/4_efficientdet/lib/src/model.py:251: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  if len(inputs) == 2:


faild onnx export


/content/Monk_Object_Detection/4_efficientdet/lib/src/utils.py:84: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  image_shape = np.array(image_shape)
/content/Monk_Object_Detection/4_efficientdet/lib/src/utils.py:96: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  anchors = torch.from_numpy(all_anchors.astype(np.float32))



faild onnx export



faild onnx export


...


faild onnx export



faild onnx export



faild onnx export



faild onnx export



faild onnx export
CPU times: user 15min 56s, sys: 3min 6s, total: 19min 2s
Wall time: 21min 10s


# Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/4_efficientdet/lib/");

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
#our trained model weights are in here in onxx format
gtf.Model(model_dir="trained/")

In [ ]:
#extract class list from our annotations
import json
with open('train/_annotations.coco.json') as json_file:
    data = json.load(json_file)
class_list = []
for category in data['categories']:
  class_list.append(category['name'])

In [ ]:
class_list

['Cars', 'bus', 'cycle', 'heavy', 'light', 'truck', 'two']

In [ ]:
%%time
test_images = [f for f in os.listdir('test') if f.endswith('.jpg')]
import random

img_path = "test/" + random.choice(test_images);

duration, scores, labels, boxes = gtf.Predict(img_path, class_list, vis_threshold=0.2);

Done. (0.262s)
NO Object Detected


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
print(scores)
print(labels)

In [ ]:
from IPython.display import Image
Image(filename='output.jpg')

# Export Trained Weights

In [ ]:
#export trained model
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%mkdir trained_export
#%cp ./trained/signatrix_efficientdet_coco.onnx ./trained_export/signatrix_efficientdet_coco_$(date +%F-%H:%M).onnx
%cp ./trained/signatrix_efficientdet_coco.pth ./trained_export/signatrix_efficientdet_coco_$(date +%F-%H:%M).pth
%mv ./trained_export/* /content/drive/My\ Drive/